In [1]:
#Load the data
#Use data_text = data[[‘headline_text’]], ‘data_text[‘index’]=data_text.index’

import pandas as pd
data = pd.read_csv('abcnews-date-text.csv', error_bad_lines=False);
data_text = data[['headline_text']]
data_text['index'] = data_text.index
documents = data_text

In [2]:
print(len(documents))
print(documents[:5])

1186018
                                       headline_text  index
0  aba decides against community broadcasting lic...      0
1     act fire witnesses must be aware of defamation      1
2     a g calls for infrastructure protection summit      2
3           air nz staff in aust strike for pay rise      3
4      air nz strike to affect australian travellers      4


In [12]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')
stemmer = PorterStemmer()

[nltk_data] Downloading package wordnet to C:\Users\Prashant
[nltk_data]     Mourya\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [13]:
#Tokenize and lemmatize the text
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result


In [14]:
doc_sample = documents[documents['index'] == 4310].values[0][0]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['ratepayers', 'group', 'wants', 'compulsory', 'local', 'govt', 'voting']


 tokenized and lemmatized document: 
['ratepay', 'group', 'want', 'compulsori', 'local', 'govt', 'vote']


In [15]:
processed_docs = documents['headline_text'].map(preprocess)
processed_docs[:10]

0               [decid, commun, broadcast, licenc]
1                               [wit, awar, defam]
2           [call, infrastructur, protect, summit]
3                      [staff, aust, strike, rise]
4             [strike, affect, australian, travel]
5               [ambiti, olsson, win, tripl, jump]
6           [antic, delight, record, break, barca]
7    [aussi, qualifi, stosur, wast, memphi, match]
8            [aust, address, secur, council, iraq]
9                         [australia, lock, timet]
Name: headline_text, dtype: object

In [16]:
#Create a dictionary as ‘dictionary=genism.corpora.Dictionary(processed_docs)’
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 broadcast
1 commun
2 decid
3 licenc
4 awar
5 defam
6 wit
7 call
8 infrastructur
9 protect
10 summit


In [17]:
#Filter extremes from dictionary
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [18]:
#Create a ‘bow_corpus’ using ‘doc2bow(doc)’ for doc in processed document
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4310]

[(162, 1), (240, 1), (292, 1), (589, 1), (839, 1), (3575, 1), (3576, 1)]

In [19]:
bow_doc_4310 = bow_corpus[4310]
for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                               dictionary[bow_doc_4310[i][0]], 
bow_doc_4310[i][1]))

Word 162 ("govt") appears 1 time.
Word 240 ("group") appears 1 time.
Word 292 ("vote") appears 1 time.
Word 589 ("local") appears 1 time.
Word 839 ("want") appears 1 time.
Word 3575 ("compulsori") appears 1 time.
Word 3576 ("ratepay") appears 1 time.


In [20]:
#From genism import corpora, models and use Tfidf for the ‘bow_corpus’
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.5860586362613176),
 (1, 0.3854657616151764),
 (2, 0.5006618583937537),
 (3, 0.5072367544211179)]


In [21]:
#Create a lda_model
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [22]:
 #Print the topics using BOW
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.027*"market" + 0.023*"school" + 0.016*"investig" + 0.015*"interview" + 0.014*"street" + 0.014*"fall" + 0.014*"student" + 0.012*"scott" + 0.011*"share" + 0.010*"week"
Topic: 1 
Words: 0.020*"donald" + 0.018*"plan" + 0.017*"canberra" + 0.015*"water" + 0.013*"feder" + 0.012*"council" + 0.010*"drum" + 0.010*"industri" + 0.010*"chang" + 0.010*"park"
Topic: 2 
Words: 0.042*"australia" + 0.037*"trump" + 0.024*"world" + 0.016*"open" + 0.015*"tasmania" + 0.014*"australian" + 0.014*"final" + 0.012*"win" + 0.011*"break" + 0.011*"record"
Topic: 3 
Words: 0.029*"queensland" + 0.021*"help" + 0.017*"bushfir" + 0.017*"miss" + 0.015*"royal" + 0.015*"tasmanian" + 0.014*"bank" + 0.013*"victoria" + 0.013*"report" + 0.012*"commiss"
Topic: 4 
Words: 0.024*"news" + 0.022*"women" + 0.021*"live" + 0.021*"warn" + 0.020*"coast" + 0.019*"health" + 0.016*"rural" + 0.015*"busi" + 0.015*"countri" + 0.013*"gold"
Topic: 5 
Words: 0.053*"polic" + 0.049*"say" + 0.024*"kill" + 0.023*"attack" + 0.021*"c

In [23]:
#Run LDA using lda_model_tfidf
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.014*"drum" + 0.010*"govern" + 0.009*"rural" + 0.009*"tuesday" + 0.008*"sport" + 0.008*"pacif" + 0.006*"august" + 0.006*"age" + 0.006*"care" + 0.006*"discuss"
Topic: 1 Word: 0.019*"news" + 0.010*"australia" + 0.009*"world" + 0.008*"final" + 0.007*"friday" + 0.007*"monday" + 0.007*"leagu" + 0.007*"nation" + 0.007*"scott" + 0.007*"test"
Topic: 2 Word: 0.025*"trump" + 0.013*"market" + 0.009*"guilti" + 0.009*"price" + 0.009*"street" + 0.008*"climat" + 0.007*"wednesday" + 0.007*"share" + 0.007*"australian" + 0.007*"thursday"
Topic: 3 Word: 0.014*"interview" + 0.008*"david" + 0.007*"east" + 0.007*"bushfir" + 0.007*"speak" + 0.006*"histori" + 0.006*"extend" + 0.006*"june" + 0.006*"abbott" + 0.006*"alan"
Topic: 4 Word: 0.011*"royal" + 0.010*"kill" + 0.008*"commiss" + 0.007*"liber" + 0.007*"christma" + 0.006*"dead" + 0.006*"crash" + 0.005*"marriag" + 0.005*"syria" + 0.005*"quiz"
Topic: 5 Word: 0.008*"juli" + 0.008*"financ" + 0.007*"insid" + 0.006*"america" + 0.006*"territori" + 

In [24]:
processed_docs[4310]

['ratepay', 'group', 'want', 'compulsori', 'local', 'govt', 'vote']

In [25]:
#Evaluate the performance by classifying sample document using LDA BOW
for index, score in sorted(lda_model[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.6183049082756042	 
Topic: 0.020*"donald" + 0.018*"plan" + 0.017*"canberra" + 0.015*"water" + 0.013*"feder" + 0.012*"council" + 0.010*"drum" + 0.010*"industri" + 0.010*"chang" + 0.010*"park"

Score: 0.15091852843761444	 
Topic: 0.013*"show" + 0.013*"liber" + 0.012*"morrison" + 0.012*"wall" + 0.011*"young" + 0.011*"christma" + 0.010*"station" + 0.010*"beach" + 0.009*"train" + 0.009*"research"

Score: 0.14322426915168762	 
Topic: 0.024*"news" + 0.022*"women" + 0.021*"live" + 0.021*"warn" + 0.020*"coast" + 0.019*"health" + 0.016*"rural" + 0.015*"busi" + 0.015*"countri" + 0.013*"gold"

Score: 0.012509899213910103	 
Topic: 0.020*"south" + 0.019*"adelaid" + 0.018*"north" + 0.016*"protest" + 0.014*"hospit" + 0.014*"labor" + 0.011*"west" + 0.011*"worker" + 0.010*"stori" + 0.010*"state"

Score: 0.012507260777056217	 
Topic: 0.034*"year" + 0.031*"charg" + 0.027*"court" + 0.025*"murder" + 0.018*"face" + 0.017*"jail" + 0.016*"accus" + 0.014*"peopl" + 0.014*"death" + 0.014*"trial"

Score: 

In [26]:
#Evaluate the performance by classifying sample document using LDA Tfidf
for index, score in sorted(lda_model_tfidf[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.5480542182922363	 
Topic: 0.013*"countri" + 0.010*"hour" + 0.009*"donald" + 0.007*"elect" + 0.007*"fund" + 0.006*"feder" + 0.006*"budget" + 0.005*"plan" + 0.005*"labor" + 0.005*"council"

Score: 0.18370500206947327	 
Topic: 0.011*"royal" + 0.010*"kill" + 0.008*"commiss" + 0.007*"liber" + 0.007*"christma" + 0.006*"dead" + 0.006*"crash" + 0.005*"marriag" + 0.005*"syria" + 0.005*"quiz"

Score: 0.1806897073984146	 
Topic: 0.008*"juli" + 0.008*"financ" + 0.007*"insid" + 0.006*"america" + 0.006*"territori" + 0.005*"newcastl" + 0.005*"central" + 0.005*"drone" + 0.005*"donat" + 0.005*"johnson"

Score: 0.012508109211921692	 
Topic: 0.025*"trump" + 0.013*"market" + 0.009*"guilti" + 0.009*"price" + 0.009*"street" + 0.008*"climat" + 0.007*"wednesday" + 0.007*"share" + 0.007*"australian" + 0.007*"thursday"

Score: 0.012508026324212551	 
Topic: 0.014*"drum" + 0.010*"govern" + 0.009*"rural" + 0.009*"tuesday" + 0.008*"sport" + 0.008*"pacif" + 0.006*"august" + 0.006*"age" + 0.006*"care" + 0.0

In [31]:
#Test the model on unseen document
unseen_document = 'The creation of the vacuum tube gave way to the first computing mechanism.'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.35030826926231384	 Topic: 0.027*"market" + 0.023*"school" + 0.016*"investig" + 0.015*"interview" + 0.014*"street"
Score: 0.18363629281520844	 Topic: 0.034*"year" + 0.031*"charg" + 0.027*"court" + 0.025*"murder" + 0.018*"face"
Score: 0.1830003261566162	 Topic: 0.036*"elect" + 0.032*"govern" + 0.025*"china" + 0.015*"rise" + 0.014*"australia"
Score: 0.18286290764808655	 Topic: 0.053*"polic" + 0.049*"say" + 0.024*"kill" + 0.023*"attack" + 0.021*"crash"
Score: 0.016699975356459618	 Topic: 0.042*"australia" + 0.037*"trump" + 0.024*"world" + 0.016*"open" + 0.015*"tasmania"
Score: 0.016699878498911858	 Topic: 0.013*"show" + 0.013*"liber" + 0.012*"morrison" + 0.012*"wall" + 0.011*"young"
Score: 0.0166997779160738	 Topic: 0.020*"donald" + 0.018*"plan" + 0.017*"canberra" + 0.015*"water" + 0.013*"feder"
Score: 0.016697514802217484	 Topic: 0.029*"queensland" + 0.021*"help" + 0.017*"bushfir" + 0.017*"miss" + 0.015*"royal"
Score: 0.016697514802217484	 Topic: 0.024*"news" + 0.022*"women" + 0.